In [1]:
# Cell 1: Imports
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Cell 2: Function to read images and labels
def read_images_and_label(folder_path):
    image_data = []
    labels = []
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                img = cv2.imread(image_path)
                image_data.append(img)
                labels.append(label)
    return np.array(image_data), np.array(labels)

In [4]:
def read_images_and_label(folder_path):
    image_data = []
    labels = []
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                try:
                    img = cv2.imread(image_path)
                    if img is not None:
                        img = cv2.resize(img, (256, 256))  # Resize images to ensure consistent shape
                        image_data.append(img)
                        labels.append(label)
                except Exception as e:
                    print(f"Error processing image: {image_path}, Error: {str(e)}")
    return np.array(image_data), np.array(labels)


In [6]:
# Cell 3: Read images and labels
folder_path = "/Users/abdulrahmankhaled/Desktop/datascience/Plant Disease Detection /plantvillage dataset/segmented"
images, labels = read_images_and_label(folder_path)

In [7]:
# Cell 4: Encoding labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [8]:
# Cell 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

In [9]:
# Cell 6: Define the model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Dropout layer added for regularization
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [10]:
# Cell 7: Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# Cell 8: Train the model
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1358/1358 [==============================] - 498s 366ms/step - loss: 3.2034 - accuracy: 0.4559 - val_loss: 1.2530 - val_accuracy: 0.6282
Epoch 2/5
1358/1358 [==============================] - 523s 385ms/step - loss: 0.8504 - accuracy: 0.7440 - val_loss: 1.1493 - val_accuracy: 0.6732
Epoch 3/5
1358/1358 [==============================] - 515s 379ms/step - loss: 0.4250 - accuracy: 0.8665 - val_loss: 1.2027 - val_accuracy: 0.6963
Epoch 4/5
1358/1358 [==============================] - 510s 376ms/step - loss: 0.3226 - accuracy: 0.8984 - val_loss: 1.4409 - val_accuracy: 0.6998
Epoch 5/5
1358/1358 [==============================] - 516s 380ms/step - loss: 0.2387 - accuracy: 0.9252 - val_loss: 1.4848 - val_accuracy: 0.6988


In [12]:
# Cell 9: Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', test_acc)


340/340 - 29s - loss: 1.4848 - accuracy: 0.6988 - 29s/epoch - 86ms/step
Test accuracy: 0.6987663507461548


In [13]:
# Cell 10: Save the model in native Keras format
model.save('segmented.keras')
